**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random as rd

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def get_epsilon(self):
        return self.epsilon
      
    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Function act allows the agent to take an action given a state s. In a training setting, a random action is taken if a randomly generated value is less than epsilon and the learned best action is taken otherwise.

In a test setting, the best learned action is taken from state s.

The introduction of epsilon allows us to overcome the problem of always taking the best learned possible option even if there is a better option out there and consequently allowed us to do epxloration. A random action is taken every now and then (when the randomly generated value i <= epsilon).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position: keeps track of the position of the rat on the board. All cells are represented as zeros except the position of the rat that is set to 1 and the borders that are set to -1

board: represents the island on which the rat moves and representes a view of the rewards. Each cell and whether it contains posion, cheese or is neutral contains -1, 0.5 or 0 respectively.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return rd.randint(1,4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        #resetting the game
        win = 0
        lose = 0
        s = env.reset() 
        game_over = False
        
        #while the game is not over
        #perform an action, update previous state and get the new state, reward and game status
        #update win/lose based on reward value
        while game_over == False: 
            s_prev = s
            action = agent.learned_act(s)
            s, reward, game_over = env.act(action)
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose + reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 4.0/-6.0. Average score (10.0)
Win/lose count 2.0/-5.0. Average score (8.5)
Win/lose count 5.5/-5.0. Average score (9.166666666666666)
Win/lose count 5.0/-4.0. Average score (9.125)
Win/lose count 0.5/-1.0. Average score (7.6)
Win/lose count 6.0/-6.0. Average score (8.333333333333334)
Win/lose count 6.5/-11.0. Average score (9.642857142857142)
Win/lose count 4.5/-9.0. Average score (10.125)
Win/lose count 5.5/-6.0. Average score (10.277777777777779)
Win/lose count 7.5/-8.0. Average score (10.8)
Win/lose count 6.0/-5.0. Average score (10.818181818181818)
Win/lose count 6.0/-9.0. Average score (11.166666666666666)
Win/lose count 2.5/-2.0. Average score (10.653846153846153)
Win/lose count 5.5/-7.0. Average score (10.785714285714286)
Win/lose count 3.5/-10.0. Average score (10.966666666666667)
Win/lose count 4.0/-7.0. Average score (10.96875)
Win/lose count 3.5/-9.0. Average score (11.058823529411764)
Win/lose count 5.5/-6.0. Average score (11.083333333333334)
Win/lose count

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__
 

\begin{equation} 
Q^\pi(s,a)=E{p^{\pi}}[\sum_{t=0}^{inf}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] 
\end{equation}
\begin{equation} 
Q^\pi(s,a)=E{p^{\pi}} [r(s_{0},a_{0})+\gamma \sum_{t=1}^{inf}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] 
\end{equation}
\begin{equation} 
Q^\pi(s,a)=r(s,a)+\gamma E{p^{\pi}} [ \sum_{t=0}^{inf}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] 
\end{equation}
\begin{equation} 
Q^\pi(s,a)=r(s,a)+ \gamma E_{(s',a')  \tilde\  p(.|s,a)} (E{p^{\pi}} [ \sum_{t=0}^{inf}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] )
\end{equation}
\begin{equation} 
Q^\pi(s,a)=E_{(s',a')  \sim\  p(.|s,a)} (r(s,a)+ \gamma  E{p^{\pi}} [ \sum_{t=0}^{inf}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] )
\end{equation}
\begin{equation} 
Q^\pi(s,a)=E_{(s',a')  \sim\  p(.|s,a)} (r(s,a)+ \gamma Q^\pi(s',a')  )
\end{equation}

optimal Q function:
\begin{equation}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) 
\end{equation}
\begin{equation}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] 
\end{equation}
\begin{equation}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] 
\end{equation}
\begin{equation}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation}

A plausible objective is one that minimizes loss:
\begin{equation}
\mathcal{L}(\theta) =  [ Q^* (s,a,\theta ) - Q(s,a,\theta)] ^2
\end{equation}
\begin{equation}
\mathcal{L}(\theta) =  (E_{s' \sim \pi^*(.|s,a)} [ ( r(s,a,s')+\gamma\max_{a'}Q^{*}(s',a', \theta)] - Q(s,a,\theta) )^2 
\end{equation}
\begin{equation}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        rand = rd.randint(0,len(self.memory)-1)
        return self.memory[rand]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #using s, predict the reward value #elaborate more here
        s = np.reshape(s,[1,5,5,self.n_state])

        reward_pred = self.model.predict(s)
        reward_max = np.argmax(reward_pred[0])
        return reward_max


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        
        for i in range(self.batch_size):
            ######## FILL IN
            target = r_
            #minibatch= self.memory.random_access()

            s_, n_s_,a_ , r_, game_over_ = self.memory.random_access()
            #s_= minibatch[0]
            #n_s_= minibatch[1]
            #a_ = minibatch[2]
            #r_= minibatch[3]
            #game_over_= minibatch[4]


            input_states[i]= s_.copy()
            ##############
            
            if game_over_:
                ######## FILL IN
                target= r_
                #######
            else:
                ######## FILL IN
                #target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                target = r_ + self.discount * max(self.model.predict(n_s_)[0])
                ##########
                


            ############
            target_int = self.model.predict(s_)

            target_int[0][a_] = target

            self.model.fit(s_, target_int, epochs=1, verbose=0)
            target_q[i]=target_int
            ##########
            
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)


        l = self.model.train_on_batch(input_states, target_q)



        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Reshape((5*5* 2, ), input_shape = (5, 5, 2)))
        model.add(Dense(256, input_dim=5*5*2,activation = "relu"))
        model.add(Dense(4, activation = "linear"))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0516 | Win/lose count 5.0/1.0 (4.0)
Epoch 001/020 | Loss 0.1709 | Win/lose count 4.5/3.0 (1.5)
Epoch 002/020 | Loss 0.1054 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/020 | Loss 0.1572 | Win/lose count 5.5/2.0 (3.5)
Epoch 004/020 | Loss 0.0367 | Win/lose count 2.5/2.0 (0.5)
Epoch 005/020 | Loss 0.1006 | Win/lose count 3.5/1.0 (2.5)
Epoch 006/020 | Loss 0.1115 | Win/lose count 5.5/2.0 (3.5)
Epoch 007/020 | Loss 0.1596 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/020 | Loss 0.1245 | Win/lose count 5.0/6.0 (-1.0)
Epoch 009/020 | Loss 0.1653 | Win/lose count 4.0/2.0 (2.0)
Epoch 010/020 | Loss 0.2188 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/020 | Loss 0.1513 | Win/lose count 7.0/4.0 (3.0)
Epoch 012/020 | Loss 0.1250 | Win/lose count 6.0/1.0 (5.0)
Epoch 013/020 | Loss 0.1269 | Win/lose count 8.5/0 (8.5)
Epoch 014/020 | Loss 0.1643 | Win/lose count 7.0/5.0 (2.0)
Epoch 015/020 | Loss 0.0814 | Win/lose count 6.5/1.0 (5.5)
Epoch 016/020 | Loss 0.1818 | Win/lose count 12.0/4.0 (8

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape = (5, 5, self.n_state), activation = 'relu'))
        model.add(Conv2D(32, (3, 3), activation = 'relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(16, activation = "relu"))
        model.add(Dense(4, activation = "linear"))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0028 | Win/lose count 1.0/6.0 (-5.0)
Epoch 001/020 | Loss 0.0098 | Win/lose count 2.0/1.0 (1.0)
Epoch 002/020 | Loss 0.0007 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/020 | Loss 0.0013 | Win/lose count 4.0/3.0 (1.0)
Epoch 004/020 | Loss 0.0193 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/020 | Loss 0.0083 | Win/lose count 3.5/0 (3.5)
Epoch 006/020 | Loss 0.0341 | Win/lose count 4.0/4.0 (0.0)
Epoch 007/020 | Loss 0.0017 | Win/lose count 5.0/1.0 (4.0)
Epoch 008/020 | Loss 0.0006 | Win/lose count 5.0/1.0 (4.0)
Epoch 009/020 | Loss 0.0230 | Win/lose count 4.0/6.0 (-2.0)
Epoch 010/020 | Loss 0.0031 | Win/lose count 6.0/3.0 (3.0)
Epoch 011/020 | Loss 0.0041 | Win/lose count 5.5/4.0 (1.5)
Epoch 012/020 | Loss 0.0030 | Win/lose count 4.5/4.0 (0.5)
Epoch 013/020 | Loss 0.0032 | Win/lose count 8.0/3.0 (5.0)
Epoch 014/020 | Loss 0.0042 | Win/lose count 4.5/0 (4.5)
Epoch 015/020 | Loss 0.0038 | Win/lose count 7.5/2.0 (5.5)
Epoch 016/020 | Loss 0.0044 | Win/lose count 4.0/4.0 (0.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.5/-1.0. Average score (3.5)
Win/lose count 1.5/0. Average score (2.5)
Win/lose count 2.5/0. Average score (2.5)
Win/lose count 2.0/0. Average score (2.375)
Win/lose count 2.0/0. Average score (2.3)
Win/lose count 6.5/-1.0. Average score (3.1666666666666665)
Win/lose count 5.5/0. Average score (3.5)
Win/lose count 0.5/0. Average score (3.125)
Win/lose count 2.0/0. Average score (3.0)
Win/lose count 5.5/0. Average score (3.25)
Win/lose count 3.0/0. Average score (3.227272727272727)
Win/lose count 1.0/0. Average score (3.0416666666666665)
Win/lose count 3.0/0. Average score (3.0384615384615383)
Win/lose count 7.0/0. Average score (3.3214285714285716)
Win/lose count 0/-1.0. Average score (3.1666666666666665)
Win/lose count 1.0/0. Average score (3.03125)
Win/lose count 5.0/-1.0. Average score (3.2058823529411766)
Win/lose count 9.5/-1.0. Average score (3.611111111111111)
Win/lose count 5.0/0. Average score (3.6842105263157894)
Win/lose count 2.0/0. Average s

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

CNN gives a better performance than FC however in both the agent barely discovers new areas which means that we have a low exploration rate.

when we changed the tempareture the final score increased mainly due to the fact that we're increasing the number of cheese and reducing that of poisonous cells.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            agent.set_epsilon(np.float(agent.get_epsilon())*0.990)
          
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
  
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
    
    def reset(self):
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                    self.board.reshape(self.grid_size, self.grid_size,1),
                                    self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


    def act(self, action, train=True):
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                      self.board.reshape(self.grid_size, self.grid_size,1),
                              self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0

        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b







In [21]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))


Epoch 000/020 | Loss 2.4280 | Win/lose count 38.99999999999993/14.399999999999967 (24.599999999999962)
Epoch 001/020 | Loss 1.2365 | Win/lose count 36.899999999999935/21.600000000000065 (15.29999999999987)
Epoch 002/020 | Loss 7.5631 | Win/lose count 79.60000000000005/0 (79.60000000000005)
Epoch 003/020 | Loss nan | Win/lose count 0.9/126.39999999999972 (-125.49999999999972)
Epoch 004/020 | Loss nan | Win/lose count 0.5/19.70000000000001 (-19.20000000000001)
Epoch 005/020 | Loss nan | Win/lose count 0/19.900000000000013 (-19.900000000000013)
Epoch 006/020 | Loss nan | Win/lose count 0/19.900000000000013 (-19.900000000000013)
Epoch 007/020 | Loss nan | Win/lose count 0/21.70000000000004 (-21.70000000000004)
Epoch 008/020 | Loss nan | Win/lose count 0.8/19.900000000000013 (-19.100000000000012)
Epoch 009/020 | Loss nan | Win/lose count 0.5/118.3999999999996 (-117.8999999999996)
Epoch 010/020 | Loss nan | Win/lose count 39.99999999999992/11.099999999999977 (28.899999999999945)
Epoch 011/02

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0/-219.99999999999926. Average score (219.99999999999926)
Win/lose count 1.0/-20.500000000000025. Average score (120.74999999999964)
Win/lose count 1.3/-116.4999999999996. Average score (119.76666666666631)
Win/lose count 39.99999999999992/-10.09999999999998. Average score (102.34999999999971)
Win/lose count 0.5/-21.300000000000033. Average score (86.23999999999977)
Win/lose count 39.199999999999925/-108.29999999999983. Average score (96.44999999999976)
Win/lose count 0.4/-118.99999999999959. Average score (99.72857142857116)
Win/lose count 0/-20.100000000000016. Average score (89.77499999999976)
Win/lose count 0.4/-22.000000000000043. Average score (82.28888888888868)
Win/lose count 40.39999999999992/-9.99999999999998. Average score (79.0999999999998)
Win/lose count 39.59999999999992/-12.199999999999973. Average score (76.61818181818161)
Win/lose count 0.4/-120.99999999999956. Average score (80.34999999999978)
Win/lose count 39.699999999999925/-107.99999999999983. Avera

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***